In [2]:
debugging = True

# Description 

# Python Import

In [3]:
from __code.__all import custom_style
custom_style.style()

import matplotlib.patches as patches
%matplotlib notebook

In [4]:
from __code import utilities, gui_widgets, file_handler, fileselector

import NeuNorm
from NeuNorm.normalization import Normalization
from NeuNorm.roi import ROI

from ipywidgets.widgets import interact
from ipywidgets import widgets
from IPython.core.display import display, HTML

ipts = utilities.get_ipts()
working_dir = utilities.get_working_dir(ipts=ipts)
if debugging:
    working_dir = '/Volumes/my_book_thunderbolt_duo/IPTS/ResonanceImaging/Brianne/'

# Select Sample Images 

In [4]:
list_images_ui = fileselector.FileSelectorPanel(instruction='Select samples images', start_dir=working_dir, multiple=True)
list_images_ui.show()

# Select OB 

In [5]:
list_ob_ui = fileselector.FileSelectorPanel(instruction='Select OB images', start_dir=working_dir, multiple=True)
list_ob_ui.show()

# Loding data 

In [6]:
list_images = list_images_ui.result_full_path()
list_obs = list_ob_ui.result_full_path()

box1 = widgets.HBox([widgets.Label("Loading Sample Progress:",
                                      layout=widgets.Layout(width='20%')),
                        widgets.IntProgress(max=len(list_images))])
display(box1)
w1 = box1.children[1]
    
box2 = widgets.HBox([widgets.Label("Loading OB Progress:",
                                      layout=widgets.Layout(width='20%')),
                        widgets.IntProgress(max=len(list_obs))])
display(box2)
w2 = box2.children[1] 

sample_array = []
for _index1, _file in enumerate(list_images):
    _data = file_handler.load_data(_file)
    sample_array.append(_data)
    w1.value = _index1 + 1

ob_array = []
for _index2, _file in enumerate(list_obs):
    _data = file_handler.load_data(_file)
    ob_array.append(_data)
    w2.value = _index2 + 1

    

In [7]:
o_norm = Normalization()
o_norm.load(data=sample_array)
o_norm.load(data=ob_array, data_type='ob')

# Preview Data

In [8]:
def plot_images(index):
    fig = plt.figure(figsize=(10,10))
    ax_img = plt.subplot(111)
    ax_img.imshow(sample_array[index], cmap='viridis')
    
preview = interact(plot_images,
                  index = widgets.IntSlider(min=0, 
                                             max=len(list_images)-1,
                                                step=1,
                                                value=0,
                                                description='Image Index',
                                           continuous_update=False))

In [9]:
def plot_obs(index):
    fig = plt.figure(figsize=(10,10))
    ax_img = plt.subplot(111)
    ax_img.imshow(ob_array[index], cmap='viridis')
    
preview = interact(plot_obs,
                  index = widgets.IntSlider(min=0, 
                                             max=len(list_obs)-1,
                                                step=1,
                                                value=0,
                                                description='Image Index',
                                           continuous_update=False))

# Select Background Region 

In [10]:
[height, width] = np.shape(sample_array[0])

def plot_roi(image_index, x_left, y_top, width, height):
    
    fig = plt.figure(figsize=(10,10))
    ax_img = plt.subplot(111)
    ax_img.imshow(sample_array[image_index],
                 cmap='viridis')
    
    _rectangle = patches.Rectangle((x_left, y_top),
                                  width,
                                  height,
                                  edgecolor = 'white',
                                  linewidth = 2,
                                  fill = False)
    ax_img.add_patch(_rectangle)
    
    return [x_left, y_top, width, height]
    
preview_bragg_edge = interact(plot_roi,
                image_index = widgets.IntSlider(min=0,
                                               max=len(list_images)-1,
                                               value=0,
                                               description='Image index:',
                                               continuous_update=False), 
                  x_left = widgets.IntSlider(min=0,
                                            max=width,
                                            step=1,
                                            value=0,
                                            description='X Left',
                                            continuous_update=False),
                  y_top = widgets.IntSlider(min=0,
                                           max=height,
                                           value=0,
                                           step=1,
                                           description='Y Top',
                                           continuous_update=False),
                  width = widgets.IntSlider(min=0,
                                           max=width,
                                           step=1,
                                           value=width-1,
                                           description="Width",
                                           continuous_update=False),
                  height = widgets.IntSlider(min=0,
                                            max=height,
                                            step=1,
                                            value=height-1,
                                            description='Height',
                                            continuous_update=False))

# Normalization

In [11]:
with_roi_ui = widgets.Checkbox(value=False,
                              description='Normalization With ROI?')
display(with_roi_ui)

In [12]:
with_roi = False
if with_roi_ui.value:
    [x_left, y_top, width_roi, height_roi] = preview_bragg_edge.widget.result
    with_roi = True
    
    _roi = ROI(x0=x_left, y0=y_top, width=width_roi, height=height_roi)
    o_norm.normalization(norm_roi=_roi)
    
else:    
    o_norm.normalization()

# Export 

In [13]:
output_folder_ui = fileselector.FileSelectorPanel(instruction='Select Output Folder', start_dir=working_dir, multiple=False, type='directory')
output_folder_ui.show()

In [14]:
output_folder = os.path.join(output_folder_ui.curdir, output_folder_ui.result(), 'normalization')
utilities.make_dir(dir=output_folder)

o_norm.export(folder=output_folder)